# Lending Tree Credit Risk

### Dependencies and data

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTEENN
from imblearn.ensemble import EasyEnsembleClassifier, BalancedRandomForestClassifier

# print(mpl.style.available)
mpl.style.use('Solarize_Light2')
%matplotlib inline

In [2]:
# Inspect top rows in data
with open('lt-credit-risk/data/loans_1q19.csv') as f:
    for i in range(5):
        print(f.readline()[:100])

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade"
"","","20000","20000","20000"," 60 months"," 17.19%","499.1","C","C5","Front desk supervisor","6 yea
"","","21225","21225","21225"," 60 months"," 14.74%","502.05","C","C2","ceo","10+ years","MORTGAGE",
"","","5000","5000","5000"," 36 months"," 17.97%","180.69","D","D1","","n/a","MORTGAGE","62000","Not


In [3]:
# Data
df = pd.read_csv('lt-credit-risk/data/loans_1q19.csv', skiprows=1, low_memory=False)
print(df.shape)
df.head(3)

(115677, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,60 months,17.19%,499.10,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,21225.0,21225.0,21225.0,60 months,14.74%,502.05,C,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000.0,5000.0,5000.0,36 months,17.97%,180.69,D,D1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


### Drop unusable data

In [4]:
# Drop rows and columns with more than 10% its values missing
df.dropna(axis=1, thresh=df.shape[0]*0.9, inplace=True) # drop cols
df.dropna(axis=0, thresh=df.shape[1]*0.9, inplace=True) # drop rows
df.shape

(115674, 97)

In [5]:
# Drop constant columns
const_cols = df.nunique()[df.nunique() < 2].index # cols w/ 1 unique val
df.drop(const_cols, axis=1, inplace=True)
df.shape

(115674, 87)

In [6]:
# Drop newly issued loans
df = df[df['loan_status'] != 'Issued']
df.shape

(96839, 87)

### Convert all columns to numeric

In [7]:
# Inspect non-numeric columns
df_num = df.copy() # make a copy
obj_cols = df_num.dtypes[df_num.dtypes == object].index
df[obj_cols].head(3)

,term,int_rate,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,next_pymnt_d,last_credit_pull_d,application_type
80,60 months,13.08%,B,B5,< 1 year,MORTGAGE,Verified,Mar-2019,Fully Paid,debt_consolidation,Debt consolidation,240xx,VA,Jul-2005,66.7%,w,NaN,Apr-2019,Individual
82,36 months,22.50%,D,D3,10+ years,RENT,Not Verified,Mar-2019,Current,debt_consolidation,Debt consolidation,957xx,CA,Aug-2002,95.6%,w,May-2019,Apr-2019,Individual
93,36 months,17.19%,C,C5,9 years,RENT,Source Verified,Mar-2019,Current,debt_consolidation,Debt consolidation,765xx,TX,Apr-2009,43.5%,w,May-2019,Apr-2019,Individual


In [8]:
""" String manipulation """

# Convert `term` to numeric
df_num['term'] = df['term'].str.replace(' months', '').astype(float)

# Convert `int_rate` to numeric
df_num['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)

# Convert `emp_length` to numeric
df_num['emp_length'] = df['emp_length'].str.extract('(\d+)').astype(float)

# Convert `revol_util` to numeric
df_num['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)

df_num[obj_cols].head(3)

,term,int_rate,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,next_pymnt_d,last_credit_pull_d,application_type
80,60.0,13.08,B,B5,1.0,MORTGAGE,Verified,Mar-2019,Fully Paid,debt_consolidation,Debt consolidation,240xx,VA,Jul-2005,66.7,w,NaN,Apr-2019,Individual
82,36.0,22.50,D,D3,10.0,RENT,Not Verified,Mar-2019,Current,debt_consolidation,Debt consolidation,957xx,CA,Aug-2002,95.6,w,May-2019,Apr-2019,Individual
93,36.0,17.19,C,C5,9.0,RENT,Source Verified,Mar-2019,Current,debt_consolidation,Debt consolidation,765xx,TX,Apr-2009,43.5,w,May-2019,Apr-2019,Individual


In [9]:
""" Datetime manipulation """

# Create a new column for `issue_d` as numeric type
df_num['issue_month'] = pd.to_datetime(df['issue_d']).dt.month

# Create a new column for `earliest_cr_line` as numeric type
df_num['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line']) # convert to dt
youngest_cr = df_num['earliest_cr_line'].max() # latest date in data
df_num['oldest_cr_age'] = (youngest_cr - df_num['earliest_cr_line']).dt.days # oldest credit age

# Create a new column for 'last_credit_pull_d' as numeric type
df_num['last_credit_pull_month'] = pd.to_datetime(df['last_credit_pull_d']).dt.month
df_num['last_credit_pull_month'] = df_num['last_credit_pull_month'].replace(12, 0) # set Dec 2018 as month 0

df_num[obj_cols].head(3)

,term,int_rate,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,next_pymnt_d,last_credit_pull_d,application_type
80,60.0,13.08,B,B5,1.0,MORTGAGE,Verified,Mar-2019,Fully Paid,debt_consolidation,Debt consolidation,240xx,VA,2005-07-01,66.7,w,NaN,Apr-2019,Individual
82,36.0,22.50,D,D3,10.0,RENT,Not Verified,Mar-2019,Current,debt_consolidation,Debt consolidation,957xx,CA,2002-08-01,95.6,w,May-2019,Apr-2019,Individual
93,36.0,17.19,C,C5,9.0,RENT,Source Verified,Mar-2019,Current,debt_consolidation,Debt consolidation,765xx,TX,2009-04-01,43.5,w,May-2019,Apr-2019,Individual


In [10]:
""" Numeric mapping """

# Convert `grade` to numeric
grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7} # num mapping
df_num['grade'] = df['grade'].map(grade_mapping).astype(float)

# Convert `home_ownership` to numeric
home_mapping = dict.fromkeys(['RENT', 'ANY', 'NONE'], 0) # num mapping
home_mapping.update(dict.fromkeys(['MORTGAGE', 'OWN'], 1)) # add 1 label
df_num['home_ownership'] = df['home_ownership'].map(home_mapping).astype(float)

# Convert `verification_state` to numeric
df_num['verification_status'] = df['verification_status'].str.replace('Source ', '') # combine verified labels
veri_mapping = {'Not Verified': 0, 'Verified': 1} # num mapping
df_num['verification_status'] = df['verification_status'].map(veri_mapping).astype(float)

# Create a new column for `initial_list_status` as numeric type
init_mapping = {'f': 0, 'w': 1} # num mapping
df_num['whole_loan'] = df['initial_list_status'].map(init_mapping).astype(float)

# Create a new column for `application_type` as numeric type
app_mapping = {'Individual': 0, 'Joint App': 1} # num mapping
df_num['joint_app'] = df['application_type'].map(app_mapping).astype(float)

# Create a new column for `loan_status` as numeric type
stat_mapping = dict.fromkeys(['Charged Off', 'In Grace Period', # num mapping
                              'Late (16-30 days)', 'Late (31-120 days)'], 1) # high risk
stat_mapping.update(dict.fromkeys(['Fully Paid', 'Current'], 0)) # low risk
df_num['high_risk'] = df['loan_status'].map(stat_mapping).astype(float)

df_num[obj_cols].head(3)

,term,int_rate,grade,sub_grade,emp_length,home_ownership,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,next_pymnt_d,last_credit_pull_d,application_type
80,60.0,13.08,2.0,B5,1.0,1.0,1.0,Mar-2019,Fully Paid,debt_consolidation,Debt consolidation,240xx,VA,2005-07-01,66.7,w,NaN,Apr-2019,Individual
82,36.0,22.50,4.0,D3,10.0,0.0,0.0,Mar-2019,Current,debt_consolidation,Debt consolidation,957xx,CA,2002-08-01,95.6,w,May-2019,Apr-2019,Individual
93,36.0,17.19,3.0,C5,9.0,0.0,NaN,Mar-2019,Current,debt_consolidation,Debt consolidation,765xx,TX,2009-04-01,43.5,w,May-2019,Apr-2019,Individual


In [11]:
""" One-hot encoding """

# Group labels into 3 categories
df_num['purpose'] = df['purpose'].replace(['debt_consolidation', 'credit_card', 'medical'], 'debt') \
                                 .replace(['home_improvement', 'car', 'house', 'vacation'], 'major_purchase') \
                                 .replace(['small_business', 'moving', 'renewable_energy', 'other'], '_other')

# One-hot encode `purpose` and drop the last label
df_num = pd.get_dummies(df_num, columns=['purpose'], drop_first=True)
df_num.head(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,total_bc_limit,total_il_high_credit_limit,issue_month,oldest_cr_age,last_credit_pull_month,whole_loan,joint_app,high_risk,purpose_debt,purpose_major_purchase
80,35000.0,35000.0,35000.0,60.0,13.08,797.80,2.0,B5,1.0,1.0,...,32700.0,130876.0,3,3867,4.0,1.0,0.0,0.0,1,0
82,20000.0,20000.0,20000.0,36.0,22.50,769.00,4.0,D3,10.0,0.0,...,47000.0,30797.0,3,4932,4.0,1.0,0.0,0.0,1,0
93,10500.0,10500.0,10500.0,36.0,17.19,375.35,3.0,C5,9.0,0.0,...,2000.0,61987.0,3,2497,4.0,1.0,0.0,0.0,1,0


### Additional cleaning

In [12]:
# Drop redundant cols
cols_to_drop = ['title', 'sub_grade', 'zip_code', 'issue_d', 'loan_status', 
                'earliest_cr_line', 'addr_state', 'verification_status', 'next_pymnt_d', 
                'last_credit_pull_d', 'initial_list_status', 'application_type']
df_num.drop(cols_to_drop, axis=1, inplace=True)
df_num.shape

(96839, 82)

In [13]:
# Drop rows with missing values
df_num.dropna(inplace=True)
df_num.shape

(84198, 82)

In [14]:
# Find highly correlated columns
cor_cols = []
for i in range(df_num.shape[1] - 1):
    for j in range(i + 1, df_num.shape[1]):
        col1 = df_num.iloc[:, i]
        col2 = df_num.iloc[:, j]
        cor = col1.corr(col2)
        if abs(cor) > 0.7:
            print(col1.name, col2.name, cor)
            if col2.name not in cor_cols:
                cor_cols.append(col2.name)
                
cor_cols

loan_amnt funded_amnt 1.0
loan_amnt funded_amnt_inv 0.999997083401125
loan_amnt installment 0.9361457315663073
loan_amnt out_prncp 0.9628116324949524
loan_amnt out_prncp_inv 0.9628298134887975
funded_amnt funded_amnt_inv 0.999997083401125
funded_amnt installment 0.9361457315663073
funded_amnt out_prncp 0.9628116324949524
funded_amnt out_prncp_inv 0.9628298134887975
funded_amnt_inv installment 0.936136708511857
funded_amnt_inv out_prncp 0.9628082995548731
funded_amnt_inv out_prncp_inv 0.9628319067457652
int_rate grade 0.9644165592939707
installment out_prncp 0.8917068406731731
installment out_prncp_inv 0.8917195745051264
open_acc total_acc 0.739408034833447
open_acc num_op_rev_tl 0.8542594490286857
open_acc num_rev_accts 0.7183869384158287
open_acc num_sats 0.9996842051086133
pub_rec pub_rec_bankruptcies 0.9997149746017331
revol_bal total_rev_hi_lim 0.7670091267372624
revol_util bc_util 0.89864728747148
revol_util percent_bc_gt_75 0.7620369638540233
total_acc num_il_tl 0.727778251003140

['funded_amnt',
 'funded_amnt_inv',
 'installment',
 'out_prncp',
 'out_prncp_inv',
 'grade',
 'total_acc',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_sats',
 'pub_rec_bankruptcies',
 'total_rev_hi_lim',
 'bc_util',
 'percent_bc_gt_75',
 'num_il_tl',
 'total_pymnt_inv',
 'total_rec_prncp',
 'last_pymnt_amnt',
 'avg_cur_bal',
 'tot_hi_cred_lim',
 'num_tl_op_past_12m',
 'open_il_24m',
 'total_bal_ex_mort',
 'total_il_high_credit_limit',
 'open_rv_24m',
 'acc_open_past_24mths',
 'bc_open_to_buy',
 'total_bc_limit',
 'oldest_cr_age',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_rev_tl_bal_gt_0',
 'num_bc_tl',
 'purpose_major_purchase']

In [15]:
# Drop highly correlated columns, keeping only 1 column per correlation
df_num.drop(cor_cols, axis=1, inplace=True)
df_num.shape

(84198, 48)

In [23]:
# Find integer columns
df_int = df_num % 1
int_cols = df_int.nunique()[df_int.nunique() < 2].index

# Convert above columns to integer type
for col in int_cols:
    df_num[col] = df_num[col].astype(int)
    
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84198 entries, 0 to 84197
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   84198 non-null  int64  
 1   term                        84198 non-null  int64  
 2   int_rate                    84198 non-null  float64
 3   emp_length                  84198 non-null  int64  
 4   home_ownership              84198 non-null  int64  
 5   annual_inc                  84198 non-null  float64
 6   dti                         84198 non-null  float64
 7   delinq_2yrs                 84198 non-null  int64  
 8   inq_last_6mths              84198 non-null  int64  
 9   open_acc                    84198 non-null  int64  
 10  pub_rec                     84198 non-null  int64  
 11  revol_bal                   84198 non-null  int64  
 12  revol_util                  84198 non-null  float64
 13  total_pymnt                 841

In [16]:
# Reset index
df_num.reset_index(drop=True, inplace=True)
df_num.head(3)

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,num_accts_ever_120_pd,num_actv_bc_tl,num_tl_90g_dpd_24m,pct_tl_nvr_dlq,issue_month,last_credit_pull_month,whole_loan,joint_app,high_risk,purpose_debt
0,35000.0,60.0,13.08,1.0,1.0,125890.0,30.48,0.0,0.0,14.0,...,0.0,5.0,0.0,100.0,3,4.0,1.0,0.0,0.0,1
1,20000.0,36.0,22.50,10.0,0.0,75000.0,24.37,0.0,0.0,8.0,...,0.0,6.0,0.0,100.0,3,4.0,1.0,0.0,0.0,1
2,10500.0,36.0,17.19,9.0,0.0,66000.0,27.24,0.0,0.0,8.0,...,0.0,1.0,0.0,85.7,3,4.0,1.0,0.0,0.0,1


In [17]:
for col in df_num.columns:
    print(col, df_num[col].nunique())
    print(df_num[col].unique())
    print()

loan_amnt 1436
[35000. 20000. 10500. ... 17850.  4375. 38675.]

term 2
[60. 36.]

int_rate 53
[13.08 22.5  17.19 20.   16.4  14.74  7.56 17.97 15.57  8.19 13.9   8.81
 11.71  7.02 10.33  6.46 12.4  11.02 25.   28.8  29.   11.8  18.94 11.31
 23.4  16.14 13.56 12.98 14.47 26.31 22.35 16.91 15.02 10.72 20.89 29.35
 28.9  28.92 28.97 28.95 24.37 19.92 25.34 27.27 30.75 29.69 30.17  6.
 30.89 30.65 28.72 30.84 30.79]

emp_length 10
[ 1. 10.  9.  8.  3.  6.  7.  2.  4.  5.]

home_ownership 2
[1. 0.]

annual_inc 6448
[125890.  75000.  66000. ...  46572.  75005. 520000.]

dti 5803
[ 30.48  24.37  27.24 ...  46.01  48.16 147.1 ]

delinq_2yrs 19
[ 0.  1.  2.  4.  6.  3.  5. 16.  7. 17.  8. 12. 14.  9. 10. 13. 11. 18.
 15.]

inq_last_6mths 6
[0. 1. 2. 3. 5. 4.]

open_acc 65
[14.  8. 17. 10.  5. 11. 27.  6.  7. 18.  9.  4. 19.  1. 29. 13. 26. 15.
 20. 16. 21. 12.  3. 25. 22. 24. 23. 32. 33. 31. 34.  2. 46. 28. 30. 35.
 52. 38. 51. 36. 37. 42. 41. 45. 43. 58. 39. 40. 54. 49. 48. 44. 56. 55.
 47. 59